In [1]:
import tensorflow as tf
import numpy as np

# 데이터 읽고 변환
data = np.loadtxt('./data.csv', delimiter=',', unpack=True, dtype='float32')

x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])

In [2]:
# 학습에 직접 사용되지는 않고 학습 횟수를 카운트하는 변수
global_step = tf.Variable(0, trainable=False, name='global_step')

In [3]:
#######
#신경망 모델 구성
#######

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_uniform([2,10], -1., 1.))
L1 = tf.nn.relu(tf.matmul(X, W1))

W2 = tf.Variable(tf.random_uniform([10,20], -1., 1.))
L2 = tf.nn.relu(tf.matmul(L1, W2))

W3 = tf.Variable(tf.random_uniform([20,3], -1., 1.))
model = tf.matmul(L2, W3)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))

optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost, global_step=global_step)

In [4]:
sess = tf.Session()
# tf.global_variables() : 앞서 정의한 변수 가져오는 함수
# 앞서 정의한 변수 모두 가져와서, 이후 파일에 저장하거나 이전 학습 결과 불러와 담는 변수로 사용
saver = tf.train.Saver(tf.global_variables())

In [5]:
# ./model 디렉터리에 기존에 학습해둔 모델 있는지 확인
# 모델 있으면 학습된값들 불러오고 아니면 변수 초기화

ckpt = tf.train.get_checkpoint_state('./model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())

INFO:tensorflow:Restoring parameters from ./model\dnn.ckpt-2


In [6]:
# 최적화 수행
for step in range(2):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    
    print('Step: %d' % sess.run(global_step), 'Cost: %.3f' % sess.run(cost, feed_dict={X: x_data, Y: y_data}))

Step: 3 Cost: 0.779
Step: 4 Cost: 0.733


In [7]:
# 최적화 끝난 뒤, 학습된 변수들 지정된 체크포인트 파일에 저장
saver.save(sess, './model/dnn.ckpt', global_step=global_step)

'./model/dnn.ckpt-4'

In [8]:
prediction = tf.argmax(model,1)
target = tf.argmax(Y,1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

예측값: [0 1 2 0 0 2]
실제값: [0 1 2 0 0 2]


In [9]:
is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy*100, feed_dict={X: x_data, Y: y_data}))

정확도: 100.00


In [10]:
#######
#텐서보드 사용
#######

# with tf.name_scope로 묶은 블록은 텐서보드에서 한 계층 내부 표현
with tf.name_scope('layer1'):
    W1 = tf.Variable(tf.random_uniform([2,10], -1., 1.), name='W1')
    L1 = tf.nn.relu(tf.matmul(X, W1))
    
with tf.name_scope('layer2'):
    W2 = tf.Variable(tf.random_uniform([10,20], -1., 1.), name='W2')
    L2 = tf.nn.relu(tf.matmul(L1, W2))
    
with tf.name_scope('output'):
    W3 = tf.Variable(tf.random_uniform([20,3], -1., 1.), name='W3')
    model = tf.matmul(L2, W3)
    
with tf.name_scope('optimizer'):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))
    
    
    optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    train_op = optimizer.minimize(cost, global_step=global_step)
    
    tf.summary.scalar('cost', cost)
    
sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())

ckpt = tf.train.get_checkpoint_state('./model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())

# 앞서 지정한 텐서들 수집    
merged = tf.summary.merge_all()
# 그래프와 텐서 값 저장할 디렉터리 설정
writer = tf.summary.FileWriter('./logs', sess.graph)

for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    
    print('Step: %d' % sess.run(global_step), 'Cost: %.3f' % sess.run(cost, feed_dict={X: x_data, Y: y_data}))
    
    summary = sess.run(merged, feed_dict={X: x_data, Y: y_data})
    writer.add_summary(summary, global_step=sess.run(global_step))
    
saver.save(sess, './model/dnn.ckpt', global_step=global_step)

prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy*100, feed_dict={X: x_data, Y: y_data}))

Step: 1 Cost: 1.632
Step: 2 Cost: 1.507
Step: 3 Cost: 1.398
Step: 4 Cost: 1.300
Step: 5 Cost: 1.222
Step: 6 Cost: 1.154
Step: 7 Cost: 1.096
Step: 8 Cost: 1.045
Step: 9 Cost: 1.002
Step: 10 Cost: 0.964
Step: 11 Cost: 0.930
Step: 12 Cost: 0.903
Step: 13 Cost: 0.879
Step: 14 Cost: 0.857
Step: 15 Cost: 0.842
Step: 16 Cost: 0.830
Step: 17 Cost: 0.817
Step: 18 Cost: 0.805
Step: 19 Cost: 0.793
Step: 20 Cost: 0.781
Step: 21 Cost: 0.769
Step: 22 Cost: 0.757
Step: 23 Cost: 0.745
Step: 24 Cost: 0.734
Step: 25 Cost: 0.723
Step: 26 Cost: 0.712
Step: 27 Cost: 0.701
Step: 28 Cost: 0.692
Step: 29 Cost: 0.683
Step: 30 Cost: 0.674
Step: 31 Cost: 0.665
Step: 32 Cost: 0.657
Step: 33 Cost: 0.649
Step: 34 Cost: 0.641
Step: 35 Cost: 0.634
Step: 36 Cost: 0.627
Step: 37 Cost: 0.621
Step: 38 Cost: 0.615
Step: 39 Cost: 0.609
Step: 40 Cost: 0.604
Step: 41 Cost: 0.599
Step: 42 Cost: 0.595
Step: 43 Cost: 0.591
Step: 44 Cost: 0.587
Step: 45 Cost: 0.584
Step: 46 Cost: 0.581
Step: 47 Cost: 0.578
Step: 48 Cost: 0.576
S